In [1]:
import os
import pandas as pd
import datetime
import re

now = datetime.datetime.now().strftime("%Y%m%d")

In [4]:
phone_df = pd.read_csv("../../crawler_data/Phone_Data_20220809.csv")
phone_df

,Brand_Name,Phone_Name,Link,Icon
0,apple,iPhone SE 3,https://www.dcfever.com/phones/specification.p...,../icon/iPhone SE 3.jpg
1,apple,iPhone 13 mini,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 mini.jpg
2,apple,iPhone 13,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13.jpg
3,apple,iPhone 13 Pro,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 Pro.jpg
4,apple,iPhone 13 Pro Max,https://www.dcfever.com/phones/specification.p...,../icon/iPhone 13 Pro Max.jpg
...,...,...,...,...
1088,sharp,Sharp SX633A,https://www.dcfever.com/phones/specification.p...,../icon/Sharp SX633A.jpg
1089,sharp,Sharp 102SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 102SH.jpg
1090,sharp,Sharp 104SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 104SH.jpg
1091,sharp,Sharp 816SH,https://www.dcfever.com/phones/specification.p...,../icon/Sharp 816SH.jpg


In [8]:
#tbl_brand.csv
count = 1
tbl_data = []
for brand in phone_df.Brand_Name.unique():
    tbl_data.append([count, brand])
    count +=1
brand_df = pd.DataFrame(data=tbl_data, columns=['id', 'Brand'])
brand_df.to_csv("../db_csv/tbl_brand.csv", index=False, encoding="utf-8")


In [5]:
#tbl_product_class
count = 1
tbl_data = []
# DSLR: 單眼, EVIL: 微單
product_class = ["NoteBook", "Tablet", "Phone", "DSLR","EVIL"]
for pc in product_class:
    tbl_data.append([count, pc])
    count +=1
product_class_df = pd.DataFrame(data=tbl_data, columns=['id', 'Product_Class'])
product_class_df.to_csv("../db_csv/tbl_product_class.csv", index=False, encoding="utf-8")

In [6]:
#tbl_source
count = 1
tbl_data = []
source = ["DCfever", "Engadget", "Mobile01", "PTT"]
for s in source:
    tbl_data.append([count, s])
    count +=1
source_df = pd.DataFrame(data=tbl_data, columns=['id','Source'])
source_df.to_csv("../db_csv/tbl_source.csv", index=False, encoding="utf-8")

In [6]:
#tbl_product
#tbl_icon

count = 1
tbl_data = []

for index, p in phone_df.iterrows():
    temp_brand_name = p['Brand_Name']
    temp_name = p['Phone_Name']
    temp_icon = p['Icon']
    temp_spec_file = "../spec_data/"+temp_brand_name+"/"+temp_name+".csv"
    tbl_data.append([index+1, temp_icon, temp_brand_name, 3, temp_name, temp_spec_file, now, now])

new_phone_df = pd.DataFrame(data=tbl_data, columns=['id','icon_id', 'brand_id', 'product_class_id', 'name', 'specification', 'create_date', 'modified_date'])

brands = pd.read_csv("../db_csv/tbl_brand.csv").iterrows()
for index, b in brands:
    temp_name = b['Brand']
    temp_id = b['id']
    new_phone_df['brand_id'] = new_phone_df['brand_id'].replace([temp_name], temp_id)

icon_df = new_phone_df[['id', 'icon_id']]
icon_df = icon_df.rename(columns={'icon_id':'link'})
for index, i in icon_df.iterrows():
    temp_icon_id = i['id']
    temp_link = i['link']
    new_phone_df['icon_id'] = new_phone_df['icon_id'].replace([temp_link], temp_icon_id)

new_phone_df.to_csv("../db_csv/tbl_product.csv", index=False, encoding="utf-8")
icon_df.to_csv("../db_csv/tbl_icon.csv", index=False, encoding="utf-8")

In [25]:
#tbl_post

#data combining
phone_list = []
all_post_df = pd.DataFrame(data = [], columns=['Post Title', 'Link', 'Date', 'Content','Phone_Name'])
crawler_data_path = "../../crawler_data/"
brand_list_dir = os.listdir(crawler_data_path)
for brand in brand_list_dir:
    if(brand != ".DS_Store" and brand != "較舊" and brand.find("csv")==-1):
        brand_data_path = crawler_data_path + brand +"/Data/"
        phone_list_dir = os.listdir(brand_data_path)
        for phone in phone_list_dir:
            if(phone != ".DS_Store"):
                phone_data_path = brand_data_path + phone +'/'
                phone_data_list_dir = os.listdir(phone_data_path)
                for data in phone_data_list_dir:
                    if(data != ".DS_Store"):
                        phone_list = []
                        temp_csv_path = phone_data_path + data
                        temp_df = pd.read_csv(temp_csv_path)
                        for i in range(len(temp_df)):
                            phone_list.append(phone)
                        temp_df['Phone_Name'] = phone_list
                        all_post_df = pd.concat([all_post_df, temp_df], axis=0)
file_name = "../../crawler_data/All_Craw_Data_" + now +".csv"
all_post_df.to_csv(file_name, index=False, encoding="utf-8")

#drop columns
all_post_df_without_nan = all_post_df.dropna(subset=['Post Title'])
all_post_df_without_nan = all_post_df_without_nan.drop(columns=['Title', 'Comment'])

#drop null data
all_post_df_without_nan = all_post_df_without_nan.dropna(axis=0)

#date fix
for index, row in all_post_df_without_nan.iterrows():
    row['Date'] = row['Date'].replace("/","-")
all_post_df_without_nan = all_post_df_without_nan.drop_duplicates(keep=False)
all_post_df_without_nan.to_csv(file_name, index=False, encoding="utf-8")
all_post_df_without_nan

,Post Title,Link,Date,Content,Phone_Name
0,iphone se3自動重啟panic full - Mobile01,https://www.mobile01.com/topicdetail.php?f=383...,2022-05-12,"['三月中買的SE3, 昨天跟今天各發生一次自動重啟了:手機有電但是螢幕關閉的情況下拿起來按...",iPhone SE 3
1,被外型耽誤的內在！ iPhone SE 3 開箱、拍攝及跑分小試- Mobile01,https://www.mobile01.com/topicdetail.php?f=383...,2022-03-14,['你預購iPhone SE 3了嗎？有考慮但還沒行動的原因在哪呢？以往iPhone SE最...,iPhone SE 3
2,第三代iPhone SE 週五即可預購：性能可比iPhone 13 但是最便宜5G ...,https://www.mobile01.com/topicdetail.php?f=383...,2022-03-09,['在今天凌晨的春季發表會上，一如預期發表了2022年新iPhone SE，並且許多規格、內...,iPhone SE 3
3,2022 iPhone 6s vs iPhone se3 - Mobile01,https://www.mobile01.com/topicdetail.php?f=383...,2022-04-29,"['IOS15.4.1的更新通知已在手機上待了一陣子,審視一番後仍是將i8與i6更新.但SE...",iPhone SE 3
4,iphone se3 下巴變窄了？ - Mobile01,https://www.mobile01.com/topicdetail.php?f=383...,2022-03-12,['看apple 官方廣告影片，發現好像新se下巴變窄了，是我錯覺嘛？htthttps://...,iPhone SE 3
...,...,...,...,...,...
6,[情報] vivo X80 Pro超大鏡頭 天璣9000+夜拍加強,https://www.ptt.cc/bbs/MobileComm/M.1649950703...,2022-04-14,Content：https://twitter.com/sondesix/status/15...,vivo X80 Pro
0,Vivo X80 Pro 相機評測：蓄勢待發,https://chinese.engadget.com//vivo-x80-pro-plu...,2022-06-01,['X80 Pro 照舊搭載了後置四攝，依然是廣角主攝、超廣角、2X 人像和 5X 遠攝的經...,vivo X80 Pro
1,"Vivo X80 在台發表，定價 NT$27,990",https://chinese.engadget.com//vivo-x80-tw-1016...,2022-05-26,"['台灣版的 X80 搭載了 6.78 吋的 FHD+（2,400 x 1,080）AMOL...",vivo X80 Pro
2,Vivo X80、X80 Pro 想靠「雙芯」上探更高定位,https://chinese.engadget.com//vivo-x80-x80-pro...,2022-04-25,['而在拍攝部分，兩款產品的自拍相機均為 32MP。X80 後置三攝由 50MP 主攝、12...,vivo X80 Pro


In [72]:
#tbl_post
id_list = []
tbl_post = pd.DataFrame(data = [], columns= ['Post Title', 'Link', 'Date', 'Content','Phone_Name', 'product_id', 'source_id'])
phone_df = pd.read_csv("../db_csv/tbl_product.csv")
source_df = pd .read_csv("../db_csv/tbl_source.csv")
all_post_df = pd.read_csv("../../crawler_data/All_Craw_Data_20220822.csv")
for i in range(len(all_post_df)):
    temp_df = all_post_df.loc[i]
    temp_phone_target = phone_df.loc[phone_df['name']==temp_df['Phone_Name']]
    temp_phone_id = temp_phone_target['id'].values[0]
    temp_link = temp_df['Link']
    for index, row in source_df.iterrows():
        if re.search(row['Source'], temp_link, re.IGNORECASE):
            temp_post_source= row['id']
    temp_df['product_id'] = temp_phone_id
    temp_df['source_id'] = temp_post_source
    new_temp_df = pd.DataFrame(data = [temp_df.tolist()], columns= ['Post Title', 'Link', 'Date', 'Content','Phone_Name', 'product_id', 'source_id'])
    tbl_post = pd.concat([tbl_post, new_temp_df], axis=0)
    id_list.append(i+1)
file_name = "../db_csv/tbl_post.csv"
tbl_post = tbl_post[['product_id', 'source_id', 'Post Title', 'Link', 'Date', 'Content','Phone_Name']]
tbl_post = tbl_post.drop(columns=['Phone_Name'])
tbl_post['id'] = id_list
tbl_post = tbl_post[['id', 'product_id', 'source_id', 'Post Title', 'Link', 'Date', 'Content']]
tbl_post.to_csv(file_name, index=False, encoding="utf-8")
tbl_post

,id,product_id,source_id,Post Title,Link,Date,Content
0,1,1,3,iphone se3自動重啟panic full - Mobile01,https://www.mobile01.com/topicdetail.php?f=383...,2022-05-12,"['三月中買的SE3, 昨天跟今天各發生一次自動重啟了:手機有電但是螢幕關閉的情況下拿起來按..."
0,2,1,3,被外型耽誤的內在！ iPhone SE 3 開箱、拍攝及跑分小試- Mobile01,https://www.mobile01.com/topicdetail.php?f=383...,2022-03-14,['你預購iPhone SE 3了嗎？有考慮但還沒行動的原因在哪呢？以往iPhone SE最...
0,3,1,3,第三代iPhone SE 週五即可預購：性能可比iPhone 13 但是最便宜5G ...,https://www.mobile01.com/topicdetail.php?f=383...,2022-03-09,['在今天凌晨的春季發表會上，一如預期發表了2022年新iPhone SE，並且許多規格、內...
0,4,1,3,2022 iPhone 6s vs iPhone se3 - Mobile01,https://www.mobile01.com/topicdetail.php?f=383...,2022-04-29,"['IOS15.4.1的更新通知已在手機上待了一陣子,審視一番後仍是將i8與i6更新.但SE..."
0,5,1,3,iphone se3 下巴變窄了？ - Mobile01,https://www.mobile01.com/topicdetail.php?f=383...,2022-03-12,['看apple 官方廣告影片，發現好像新se下巴變窄了，是我錯覺嘛？htthttps://...
...,...,...,...,...,...,...,...
0,3322,923,4,[情報] vivo X80 Pro超大鏡頭 天璣9000+夜拍加強,https://www.ptt.cc/bbs/MobileComm/M.1649950703...,2022-04-14,Content：https://twitter.com/sondesix/status/15...
0,3323,923,2,Vivo X80 Pro 相機評測：蓄勢待發,https://chinese.engadget.com//vivo-x80-pro-plu...,2022-06-01,['X80 Pro 照舊搭載了後置四攝，依然是廣角主攝、超廣角、2X 人像和 5X 遠攝的經...
0,3324,923,2,"Vivo X80 在台發表，定價 NT$27,990",https://chinese.engadget.com//vivo-x80-tw-1016...,2022-05-26,"['台灣版的 X80 搭載了 6.78 吋的 FHD+（2,400 x 1,080）AMOL..."
0,3325,923,2,Vivo X80、X80 Pro 想靠「雙芯」上探更高定位,https://chinese.engadget.com//vivo-x80-x80-pro...,2022-04-25,['而在拍攝部分，兩款產品的自拍相機均為 32MP。X80 後置三攝由 50MP 主攝、12...


In [7]:
#tbl_label
label_list = ["預算","晶片","螢幕","電池","鏡頭","容量"]
count = 1
tbl_data = []
for label in label_list:
    tbl_data.append([count,3,1093+count,label])
    count+=1

label_df = pd.DataFrame(data=tbl_data, columns=['id','product_class_id','icon_id','name'])
label_df.to_csv("../db_csv/tbl_label.csv", index=False, encoding="utf-8")

In [25]:
#tbl_label_detail
count = 1
tbl_data = []
label_class_dict = {"budget":1, "chip": 2, "screen": 3, "battery": 4, "lens": 5, "capacity": 6}
path_of_filter_csv = "../../word_similarity/Filtering/"
label_class_list = os.listdir(path_of_filter_csv)

for label_class_folder in label_class_list:
    label_class = label_class_folder[7:]
    label_id = label_class_dict[label_class]
    label_product_folder_list = os.listdir(path_of_filter_csv + label_class_folder)
    for label_files in label_product_folder_list:
        product_id = label_files[25:-4]
        detail_path = path_of_filter_csv + label_class_folder + "/" + label_files
        label_detail_csv = pd.read_csv(detail_path)
        label_detail_list = label_detail_csv['Word'].tolist()
        for word in label_detail_list:
            tbl_data.append([count, product_id, label_id, word, now, now])
            count+=1

label_detail_df = pd.DataFrame(data=tbl_data, columns=['id','product_id','label_id','content', 'create_date', 'modified_date'])
label_detail_df.to_csv("../db_csv/tbl_label_detail.csv", index=False, encoding="utf-8")
